# Beijing air quality forecasting

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

# For deep learning - we'll use TensorFlow/Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, GRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("Libraries imported successfully!")
print(f"TensorFlow version: {tf.__version__}")

Libraries imported successfully!
TensorFlow version: 2.20.0


In [ ]:
# Load the data
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

print("Data loaded successfully!")
print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")

Data loaded successfully!
Train shape: (30676, 12)
Test shape: (13148, 11)


# Data exploration

Explore the dataset with statistics and visualizations to understand the data better.

In [19]:
# Basic data exploration
print("Train data info:")
print(train.info())
print("\nTrain data describe:")
print(train.describe())

print("\nMissing values:")
print("Train missing values:")
print(train.isnull().sum())
print("\nTest missing values:")
print(test.isnull().sum())

Train data info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30676 entries, 0 to 30675
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   No        30676 non-null  int64  
 1   DEWP      30676 non-null  float64
 2   TEMP      30676 non-null  float64
 3   PRES      30676 non-null  float64
 4   Iws       30676 non-null  float64
 5   Is        30676 non-null  float64
 6   Ir        30676 non-null  float64
 7   datetime  30676 non-null  object 
 8   cbwd_NW   30676 non-null  float64
 9   cbwd_SE   30676 non-null  float64
 10  cbwd_cv   30676 non-null  float64
 11  pm2.5     28755 non-null  float64
dtypes: float64(10), int64(1), object(1)
memory usage: 2.8+ MB
None

Train data describe:
                 No          DEWP          TEMP          PRES           Iws  \
count  30676.000000  30676.000000  30676.000000  30676.000000  30676.000000   
mean   15338.500000     -0.029431     -0.062712      0.013612      0.030542   
s

In [20]:
# Inspecting the first few rows of the dataset to understand its structure.
print("Training Data Overview:")
train.head()
print(train.tail())
print(f"First datetime: {train['datetime'].iloc[0]}")
print(f"Last datetime: {train['datetime'].iloc[-1]}")

Training Data Overview:
          No      DEWP      TEMP      PRES       Iws        Is        Ir  \
30671  30672  1.467633  0.946961 -2.088668 -0.415099 -0.069353  2.687490   
30672  30673  1.329064  0.864984 -2.186052 -0.379306 -0.069353  3.393779   
30673  30674  1.259780  0.701029 -2.088668 -0.263130 -0.069353  4.100068   
30674  30675  1.190496  0.701029 -2.088668 -0.146953 -0.069353  4.806358   
30675  30676  1.190496  0.701029 -2.186052 -0.084366 -0.069353 -0.137667   

                  datetime   cbwd_NW   cbwd_SE   cbwd_cv  pm2.5  
30671  2013-07-01 23:00:00 -0.690542 -0.732019 -0.522096   50.0  
30672  2013-07-02 00:00:00  1.448138 -0.732019 -0.522096   41.0  
30673  2013-07-02 01:00:00  1.448138 -0.732019 -0.522096   32.0  
30674  2013-07-02 02:00:00  1.448138 -0.732019 -0.522096   19.0  
30675  2013-07-02 03:00:00  1.448138 -0.732019 -0.522096   18.0  
First datetime: 2010-01-01 00:00:00
Last datetime: 2013-07-02 03:00:00


In [21]:
train.columns

Index(['No', 'DEWP', 'TEMP', 'PRES', 'Iws', 'Is', 'Ir', 'datetime', 'cbwd_NW',
       'cbwd_SE', 'cbwd_cv', 'pm2.5'],
      dtype='object')

In [22]:
# Ensure 'datetime' column is in datetime format
train['datetime'] = pd.to_datetime(train['datetime'])

test['datetime'] = pd.to_datetime(test['datetime'])

# Set the 'datetime' column as the index for better time-series handling
train.set_index('datetime', inplace=True)
# val.set_index('datetime', inplace=True)
test.set_index('datetime', inplace=True)

print("Datetime preprocessing completed!")
print(f"Train date range: {train.index.min()} to {train.index.max()}")
print(f"Test date range: {test.index.min()} to {test.index.max()}")


Datetime preprocessing completed!
Train date range: 2010-01-01 00:00:00 to 2013-07-02 03:00:00
Test date range: 2013-07-02 04:00:00 to 2014-12-31 23:00:00


In [23]:
# Examine PM2.5 values
print("PM2.5 analysis")
print(f"PM2.5 min: {train['pm2.5'].min()}")
print(f"PM2.5 max: {train['pm2.5'].max()}")
print(f"PM2.5 mean: {train['pm2.5'].mean():.2f}")
print(f"PM2.5 std: {train['pm2.5'].std():.2f}")

# Check where missing values are located
print(f"\nMissing PM2.5 pattern")
missing_count = train['pm2.5'].isnull().sum()
print(f"Missing values: {missing_count}")
print(f"Percentage missing: {missing_count/len(train)*100:.2f}%")


PM2.5 analysis
PM2.5 min: 0.0
PM2.5 max: 994.0
PM2.5 mean: 100.79
PM2.5 std: 93.14

Missing PM2.5 pattern
Missing values: 1921
Percentage missing: 6.26%


In [24]:
# Check the missing value pattern more thoroughly
print("Detailed Missing PM2.5 Analysis")
print("First 20 rows of missing PM2.5:")
missing_mask = train['pm2.5'].isnull()
print(train[missing_mask].head(20)[['pm2.5']])

# Find where valid data starts
first_valid_idx = train['pm2.5'].first_valid_index()
print(f"\nFirst valid PM2.5 measurement: {first_valid_idx}")

# Check if missing values are scattered or concentrated
print(f"\nMissing values by month:")
train_with_month = train.copy()
train_with_month['month'] = train_with_month.index.month
print(train_with_month.groupby('month')['pm2.5'].apply(lambda x: x.isnull().sum()))

Detailed Missing PM2.5 Analysis
First 20 rows of missing PM2.5:
                     pm2.5
datetime                  
2010-01-01 00:00:00    NaN
2010-01-01 01:00:00    NaN
2010-01-01 02:00:00    NaN
2010-01-01 03:00:00    NaN
2010-01-01 04:00:00    NaN
2010-01-01 05:00:00    NaN
2010-01-01 06:00:00    NaN
2010-01-01 07:00:00    NaN
2010-01-01 08:00:00    NaN
2010-01-01 09:00:00    NaN
2010-01-01 10:00:00    NaN
2010-01-01 11:00:00    NaN
2010-01-01 12:00:00    NaN
2010-01-01 13:00:00    NaN
2010-01-01 14:00:00    NaN
2010-01-01 15:00:00    NaN
2010-01-01 16:00:00    NaN
2010-01-01 17:00:00    NaN
2010-01-01 18:00:00    NaN
2010-01-01 19:00:00    NaN

First valid PM2.5 measurement: 2010-01-02 00:00:00

Missing values by month:
month
1     242
2       8
3     163
4     192
5     109
6     178
7      59
8     363
9     259
10    134
11     83
12    131
Name: pm2.5, dtype: int64


# Handle missing values

Check the dataset for missing values and decide how to handle them.

In [25]:
# Step 1: Check the gaps more systematically
print("Gap analysis")
# Find consecutive missing periods
missing_periods = train['pm2.5'].isnull()
# Group consecutive missing values
groups = (missing_periods != missing_periods.shift()).cumsum()
missing_groups = train[missing_periods].groupby(groups).size()

print("Consecutive missing value periods:")
print(f"Number of missing periods: {len(missing_groups)}")
print(f"Average gap length: {missing_groups.mean():.1f} hours")
print(f"Max gap length: {missing_groups.max()} hours")
print(f"Min gap length: {missing_groups.min()} hours")

Gap analysis
Consecutive missing value periods:
Number of missing periods: 163
Average gap length: 11.8 hours
Max gap length: 155 hours
Min gap length: 1 hours


In [26]:
# Step 2: Handle missing values
print("Handling Missing Values")
print(f"Original data points: {len(train)}")
print(f"Missing PM2.5: {train['pm2.5'].isnull().sum()}")

# Simple dropna
train_clean = train.dropna(subset=['pm2.5']).copy()
print(f"After dropping missing PM2.5: {len(train_clean)}")
print(f"Data retained: {len(train_clean)/len(train)*100:.1f}%")

# Verify no missing values remain
print(f"Missing values after cleaning: {train_clean.isnull().sum().sum()}")

Handling Missing Values
Original data points: 30676
Missing PM2.5: 1921
After dropping missing PM2.5: 28755
Data retained: 93.7%
Missing values after cleaning: 0


In [27]:
# Check for time continuity (important for time series)
print("Time continuity check:")
time_diff = train_clean.index.to_series().diff()
expected_freq = pd.Timedelta(hours=1)

# Find any gaps larger than 1 hour
gaps = time_diff[time_diff > expected_freq]
print(f"Number of gaps > 1 hour: {len(gaps)}")
if len(gaps) > 0:
    print("Largest gaps:")
    print(gaps.nlargest(5))
else:
    print("No gaps found - data is continuous!")

Time continuity check:
Number of gaps > 1 hour: 162
Largest gaps:
datetime
2010-09-27 16:00:00   6 days 12:00:00
2012-12-28 13:00:00   5 days 08:00:00
2011-10-07 16:00:00   4 days 04:00:00
2011-03-21 16:00:00   3 days 20:00:00
2010-09-30 21:00:00   3 days 05:00:00
Name: datetime, dtype: timedelta64[ns]


## Feature engineering

In [ ]:
# FINAL OPTIMIZED FEATURE ENGINEERING
def create_final_optimized_features(df, target_col='pm2.5', has_target=True):
    """Create the most predictive features for final optimization"""
    df = df.copy()

    # Enhanced temporal features
    df['hour'] = df.index.hour
    df['day_of_week'] = df.index.dayofweek
    df['month'] = df.index.month
    df['day_of_year'] = df.index.dayofyear

    # More sophisticated cyclical encoding
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['day_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['day_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

    # Add pollution-specific patterns
    df['is_winter'] = ((df['month'] >= 11) | (df['month'] <= 2)).astype(int)
    df['is_heating_season'] = ((df['month'] >= 11) | (df['month'] <= 3)).astype(int)
    df['is_rush_hour'] = ((df['hour'] >= 7) & (df['hour'] <= 9) |
                         (df['hour'] >= 17) & (df['hour'] <= 19)).astype(int)

    # Critical lag features
    critical_lags = [1, 2, 3, 6, 12]
    key_features = ['DEWP', 'TEMP', 'PRES', 'Iws']

    for lag in critical_lags:
        for col in key_features:
            df[f'{col}_lag_{lag}'] = df[col].shift(lag)

    # Enhanced moving averages with volatility
    for window in [3, 6, 12]:
        for col in key_features:
            df[f'{col}_ma_{window}'] = df[col].rolling(window=window, min_periods=1).mean()
            df[f'{col}_std_{window}'] = df[col].rolling(window=window, min_periods=1).std()

    # Advanced interaction features
    df['temp_dewp_diff'] = df['TEMP'] - df['DEWP']
    df['pressure_wind_interaction'] = df['PRES'] * df['Iws']
    df['humidity_proxy'] = df['DEWP'] / (df['TEMP'] + 1e-8)
    df['wind_stability'] = 1 / (df[['cbwd_NW', 'cbwd_SE', 'cbwd_cv']].std(axis=1) + 1e-8)

    # Fill NaN and clean up
    df = df.bfill().ffill()
    df = df.drop(['hour', 'day_of_week', 'month', 'day_of_year'], axis=1, errors='ignore')

    return df

# Advanced outlier treatment
Q1 = train_clean['pm2.5'].quantile(0.25)
Q3 = train_clean['pm2.5'].quantile(0.75)
IQR = Q3 - Q1
upper_bound = Q3 + 2 * IQR

print(f"Advanced outlier capping at {upper_bound:.1f}")
train_treated = train_clean.copy()
train_treated['pm2.5'] = np.clip(train_treated['pm2.5'], 0, upper_bound)

# Apply final feature engineering
train_enhanced = create_final_optimized_features(train_treated, 'pm2.5', has_target=True)
test_enhanced = create_final_optimized_features(test, 'pm2.5', has_target=False)

print(f"Final optimized features: {len(train_enhanced.columns)}")

Advanced outlier capping at 368.0
Final optimized features: 68


# Feature and target separation

Separate features and target variable for model training.

In [ ]:
# UPDATED FEATURE ANALYSIS FOR ENHANCED FEATURES
print("Enhanced Features Analysis")
print("Columns in enhanced training data:")
print(train_enhanced.columns.tolist())
print(f"\nEnhanced data shape: {train_enhanced.shape}")
print(f"Original shape was: {train_clean.shape}")
print(f"Feature increase: +{len(train_enhanced.columns) - len(train_clean.columns)} features")

# Look at feature correlations with PM2.5 for enhanced features
print("\nTop Feature Correlations with PM2.5 (Enhanced)")
correlations = train_enhanced.corr()['pm2.5'].sort_values(key=abs, ascending=False)
print("Top 15 most correlated features:")
print(correlations.head(15))

# Show new feature types
print(f"\nNew feature categories added:")
new_features = [col for col in train_enhanced.columns if col not in train_clean.columns]
print(f"- Temporal features: {len([f for f in new_features if any(x in f for x in ['sin', 'cos', 'hour', 'winter', 'rush'])])}")
print(f"- Lag features: {len([f for f in new_features if 'lag' in f])}")
print(f"- Moving averages: {len([f for f in new_features if 'ma_' in f])}")
print(f"- Volatility features: {len([f for f in new_features if 'std_' in f])}")
print(f"- Interaction features: {len([f for f in new_features if any(x in f for x in ['diff', 'interaction', 'proxy', 'stability'])])}")

print(f"\nTotal new features: {len(new_features)}")

Enhanced Features Analysis
Columns in enhanced training data:
['No', 'DEWP', 'TEMP', 'PRES', 'Iws', 'Is', 'Ir', 'cbwd_NW', 'cbwd_SE', 'cbwd_cv', 'pm2.5', 'hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'month_sin', 'month_cos', 'is_winter', 'is_heating_season', 'is_rush_hour', 'DEWP_lag_1', 'TEMP_lag_1', 'PRES_lag_1', 'Iws_lag_1', 'DEWP_lag_2', 'TEMP_lag_2', 'PRES_lag_2', 'Iws_lag_2', 'DEWP_lag_3', 'TEMP_lag_3', 'PRES_lag_3', 'Iws_lag_3', 'DEWP_lag_6', 'TEMP_lag_6', 'PRES_lag_6', 'Iws_lag_6', 'DEWP_lag_12', 'TEMP_lag_12', 'PRES_lag_12', 'Iws_lag_12', 'DEWP_ma_3', 'DEWP_std_3', 'TEMP_ma_3', 'TEMP_std_3', 'PRES_ma_3', 'PRES_std_3', 'Iws_ma_3', 'Iws_std_3', 'DEWP_ma_6', 'DEWP_std_6', 'TEMP_ma_6', 'TEMP_std_6', 'PRES_ma_6', 'PRES_std_6', 'Iws_ma_6', 'Iws_std_6', 'DEWP_ma_12', 'DEWP_std_12', 'TEMP_ma_12', 'TEMP_std_12', 'PRES_ma_12', 'PRES_std_12', 'Iws_ma_12', 'Iws_std_12', 'temp_dewp_diff', 'pressure_wind_interaction', 'humidity_proxy', 'wind_stability']

Enhanced data shape: (28755, 68)
Or

In [30]:

print("Enhanced Feature-target split")

# Get feature columns that exist in BOTH train and test
train_feature_cols = [col for col in train_enhanced.columns if col not in ['pm2.5', 'No']]
test_feature_cols = [col for col in test_enhanced.columns if col not in ['No']]

# Use only features that exist in both datasets
common_features = list(set(train_feature_cols) & set(test_feature_cols))
print(f"Features in train only: {len(train_feature_cols)}")
print(f"Features in test only: {len(test_feature_cols)}")
print(f"Common features: {len(common_features)}")

# Features that exist in train but not test (PM2.5 lag features)
train_only_features = list(set(train_feature_cols) - set(test_feature_cols))
if train_only_features:
    print(f"Train-only features (will be excluded): {train_only_features}")

# Use common features for modeling
feature_cols = common_features

X_train_full = train_enhanced[feature_cols].values
y_train_full = train_enhanced['pm2.5'].values
X_test_full = test_enhanced[feature_cols].values

print(f"Final feature count: {len(feature_cols)}")
print(f"X_train shape: {X_train_full.shape}")
print(f"y_train shape: {y_train_full.shape}")
print(f"X_test shape: {X_test_full.shape}")

# Check for any remaining NaN values
print(f"NaN in X_train: {np.isnan(X_train_full).sum()}")
print(f"NaN in y_train: {np.isnan(y_train_full).sum()}")
print(f"NaN in X_test: {np.isnan(X_test_full).sum()}")

Enhanced Feature-target split
Features in train only: 66
Features in test only: 66
Common features: 66
Final feature count: 66
X_train shape: (28755, 66)
y_train shape: (28755,)
X_test shape: (13148, 66)
NaN in X_train: 0
NaN in y_train: 0
NaN in X_test: 0


In [31]:
# IMPROVED SCALING (RobustScaler is better for time series with outliers)
from sklearn.preprocessing import RobustScaler

print("Enhanced scaling with RobustScaler")
scaler_X = RobustScaler()  # Less sensitive to outliers than StandardScaler
scaler_y = StandardScaler()

# Fit and transform
X_train_scaled = scaler_X.fit_transform(X_train_full)
X_test_scaled = scaler_X.transform(X_test_full)
y_train_scaled = scaler_y.fit_transform(y_train_full.reshape(-1, 1)).flatten()

print(f"X_train_scaled shape: {X_train_scaled.shape}")
print(f"X_test_scaled shape: {X_test_scaled.shape}")
print(f"y_train_scaled shape: {y_train_scaled.shape}")

print("Scaling completed with RobustScaler!")

Enhanced scaling with RobustScaler
X_train_scaled shape: (28755, 66)
X_test_scaled shape: (13148, 66)
y_train_scaled shape: (28755,)
Scaling completed with RobustScaler!


In [ ]:
# SIMPLIFIED SEQUENCE LENGTH OPTIMIZATION
def create_sequences_improved(X, y, sequence_length):
    """Create sequences with better memory efficiency"""
    X_seq, y_seq = [], []

    for i in range(sequence_length, len(X)):
        X_seq.append(X[i-sequence_length:i])
        y_seq.append(y[i])

    return np.array(X_seq), np.array(y_seq)

# Set optimal sequence length (36 works well based on testing)
SEQUENCE_LENGTH = 36
print(f"Using optimized sequence length: {SEQUENCE_LENGTH} hours")

# Create training sequences with optimized length
X_train_seq, y_train_seq = create_sequences_improved(X_train_scaled, y_train_scaled, SEQUENCE_LENGTH)

print(f"Optimized sequence shapes:")
print(f"  X_train_seq: {X_train_seq.shape}")
print(f"  y_train_seq: {y_train_seq.shape}")
print(f"Samples lost due to sequence creation: {len(X_train_scaled) - len(X_train_seq)}")
print(f"Remaining training samples: {len(X_train_seq)}")

Using optimized sequence length: 36 hours
Optimized sequence shapes:
  X_train_seq: (28719, 36, 66)
  y_train_seq: (28719,)
Samples lost due to sequence creation: 36
Remaining training samples: 28719


In [33]:
# FINAL OPTIMIZED SPLIT (More training data)
print("Final optimized train-validation split")

# Use only 12% for validation (was 15%) = maximum training data
val_size = 0.12
split_idx = int(len(X_train_seq) * (1 - val_size))

X_train_final = X_train_seq[:split_idx]
X_val = X_train_seq[split_idx:]
y_train_final = y_train_seq[:split_idx]
y_val = y_train_seq[split_idx:]

print(f"Final training samples: {len(X_train_final)} (maximized)")
print(f"Final validation samples: {len(X_val)} (minimized)")
print(f"Training data ratio: {(1-val_size)*100:.0f}%")

Final optimized train-validation split
Final training samples: 25272 (maximized)
Final validation samples: 3447 (minimized)
Training data ratio: 88%


# Model building

Build and train LSTM model for time series forecasting.

In [ ]:
# ENSEMBLE OF DIVERSE MODELS
from tensorflow.keras.layers import Bidirectional

def create_diverse_ensemble(input_shape):
    """Create 4 diverse simple models"""

    # Model 1: Standard LSTM
    model1 = Sequential([
        LSTM(48, return_sequences=False, input_shape=input_shape),
        Dense(24, activation='relu'),
        Dropout(0.2),
        Dense(1)
    ])

    # Model 2: GRU
    model2 = Sequential([
        GRU(48, return_sequences=False, input_shape=input_shape),
        Dense(24, activation='relu'),
        Dropout(0.2),
        Dense(1)
    ])

    # Model 3: Bidirectional LSTM
    model3 = Sequential([
        Bidirectional(LSTM(24, return_sequences=False, input_shape=input_shape)),
        Dense(24, activation='relu'),
        Dropout(0.2),
        Dense(1)
    ])

    # Model 4: Deep narrow
    model4 = Sequential([
        LSTM(32, return_sequences=True, input_shape=input_shape),
        LSTM(24, return_sequences=False),
        Dense(16, activation='relu'),
        Dense(1)
    ])

    # Different learning rates for diversity
    lrs = [0.002, 0.0015, 0.0025, 0.002]
    models = [model1, model2, model3, model4]

    for i, (model, lr) in enumerate(zip(models, lrs)):
        model.compile(optimizer=Adam(learning_rate=lr, clipnorm=1.0),
                     loss='mse', metrics=['mae'])
        print(f"Model {i+1}: {model.layers[0].__class__.__name__} (LR: {lr})")

    return models

# Create ensemble
print("Creating diverse ensemble...")
input_shape = (SEQUENCE_LENGTH, X_train_final.shape[2])
ensemble_models = create_diverse_ensemble(input_shape)

Creating diverse ensemble...
Model 1: LSTM (LR: 0.002)
Model 2: GRU (LR: 0.0015)
Model 3: Bidirectional (LR: 0.0025)
Model 4: LSTM (LR: 0.002)


In [ ]:
# TRAIN ENSEMBLE WITH DIFFERENT STRATEGIES
print("Training diverse ensemble...")

trained_models = []
individual_rmses = []

for i, model in enumerate(ensemble_models):
    print(f"Training model {i+1}/4...")

    # Different patience for each model
    early_stop = EarlyStopping(monitor='val_loss', patience=20-i*2, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=8-i, min_lr=1e-7)

    history = model.fit(
        X_train_final, y_train_final,
        validation_data=(X_val, y_val),
        epochs=100,
        batch_size=32,
        callbacks=[early_stop, reduce_lr],
        verbose=0
    )

    # Evaluate individual model
    val_pred_scaled = model.predict(X_val, verbose=0)
    val_pred_original = scaler_y.inverse_transform(val_pred_scaled.reshape(-1, 1)).flatten()
    val_true_original = scaler_y.inverse_transform(y_val.reshape(-1, 1)).flatten()
    rmse = np.sqrt(np.mean((val_true_original - val_pred_original)**2))

    print(f"Model {i+1} RMSE: {rmse:.2f}")
    individual_rmses.append(rmse)
    trained_models.append(model)

print(f"Individual model RMSEs: {[f'{r:.2f}' for r in individual_rmses]}")

Training diverse ensemble...
Training model 1/4...
Model 1 RMSE: 52.78
Training model 2/4...
Model 2 RMSE: 53.08
Training model 3/4...
Model 3 RMSE: 56.00
Training model 4/4...
Model 4 RMSE: 57.95
Individual model RMSEs: ['52.78', '53.08', '56.00', '57.95']


In [36]:
# WEIGHTED ENSEMBLE PREDICTIONS
print("Creating weighted ensemble predictions...")

# Calculate performance-based weights
inv_rmses = [1/rmse for rmse in individual_rmses]
weights = [w/sum(inv_rmses) for w in inv_rmses]
print(f"Model weights: {[f'{w:.3f}' for w in weights]}")

# Validation ensemble
val_predictions = []
for model in trained_models:
    pred_scaled = model.predict(X_val, verbose=0)
    pred_original = scaler_y.inverse_transform(pred_scaled.reshape(-1, 1)).flatten()
    val_predictions.append(pred_original)

val_ensemble = np.average(val_predictions, axis=0, weights=weights)
val_true_original = scaler_y.inverse_transform(y_val.reshape(-1, 1)).flatten()
final_rmse = np.sqrt(np.mean((val_true_original - val_ensemble)**2))

print(f"FINAL ENSEMBLE PERFORMANCE:")
print(f"  Validation RMSE: {final_rmse:.2f}")
print(f"  Improvement from 69.67: {69.67 - final_rmse:.2f} points")
print(f"  Target < 65: {'ACHIEVED!' if final_rmse < 65 else 'Very close!'}")
print(f"  Estimated Kaggle score: ~{final_rmse * 100:.0f}")

Creating weighted ensemble predictions...
Model weights: ['0.260', '0.258', '0.245', '0.237']
FINAL ENSEMBLE PERFORMANCE:
  Validation RMSE: 51.75
  Improvement from 69.67: 17.92 points
  Target < 65: ACHIEVED!
  Estimated Kaggle score: ~5175


In [39]:
# FINAL ENSEMBLE TEST PREDICTIONS
import datetime

print("Creating final ensemble test predictions...")

# First, create test sequences (this was missing!)
print("Creating test sequences...")
last_train_X = X_train_scaled[-SEQUENCE_LENGTH:]
combined_X = np.vstack([last_train_X, X_test_scaled])

print(f"Last training samples: {last_train_X.shape}")
print(f"Combined data shape: {combined_X.shape}")

# Create test sequences
X_test_sequences = []
for i in range(SEQUENCE_LENGTH, len(combined_X)):
    X_test_sequences.append(combined_X[i-SEQUENCE_LENGTH:i])

X_test_sequences = np.array(X_test_sequences)
print(f"Test sequences shape: {X_test_sequences.shape}")

# Now make predictions from each model
print("Making ensemble predictions...")
test_predictions = []
for model in trained_models:
    pred_scaled = model.predict(X_test_sequences, verbose=0)
    pred_original = scaler_y.inverse_transform(pred_scaled.reshape(-1, 1)).flatten()
    test_predictions.append(pred_original)

# Weighted ensemble for test
test_ensemble = np.average(test_predictions, axis=0, weights=weights)
test_ensemble = np.maximum(test_ensemble, 0)  # No negative values

print(f"Final test prediction statistics:")
print(f"  Min: {test_ensemble.min():.1f}")
print(f"  Max: {test_ensemble.max():.1f}")
print(f"  Mean: {test_ensemble.mean():.1f}")

# Create submission with detailed naming
def format_datetime_no_leading_zero(dt):
    return f"{dt.year}-{dt.month:02d}-{dt.day:02d} {dt.hour}:{dt.minute:02d}:{dt.second:02d}"

row_ids_formatted = [format_datetime_no_leading_zero(dt) for dt in test.index]

submission_final = pd.DataFrame({
    'row ID': row_ids_formatted,
    'pm2.5': test_ensemble.astype(int)
})

# Generate detailed filename
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
rmse_str = f"{final_rmse:.2f}".replace(".", "_")
submission_path = f'FINAL_submission_RMSE_{rmse_str}_ensemble_{timestamp}.csv'

submission_final.to_csv(submission_path, index=False)

print(f"FINAL submission saved as: {submission_path}")
print(f"Expected performance: RMSE {final_rmse:.2f})")
print(f"Target achieved: {'YES! Below 4000!' if final_rmse < 67.0 else 'Very close to target!'}")

Creating final ensemble test predictions...
Creating test sequences...
Last training samples: (36, 66)
Combined data shape: (13184, 66)
Test sequences shape: (13148, 36, 66)
Making ensemble predictions...
Final test prediction statistics:
  Min: 2.5
  Max: 385.7
  Mean: 95.6
FINAL submission saved as: FINAL_submission_RMSE_51_75_ensemble_20250921_153727.csv
Expected performance: RMSE 51.75)
Target achieved: YES! Below 4000!
